In [1]:
from datetime import datetime, timezone
import sys
import json
sys.path.append('..')
import os
os.chdir('./..')
from connectors.msgraph import Msgraph
from connectors.elk import Elk
mg = Msgraph()
elk = Elk()
es = elk.es

MsGraph: token created
Elasticsearch: Connection sucessful


In [2]:
def fetch_all_docs(index, file_name):
    # Initialize the scroll
    page = es.search(
        index=index,
        scroll='2m',  # Keep the scroll context for 2 minutes
        size=1000,  # Number of results per "page"
        body={
            "query": {
                "match_all": {}  # Match all query to retrieve all documents
            }
        }
    )
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']['value']
    # Start scrolling
    while scroll_size > 0:
        "Scrolling..."
        # Before writing to the file, you can process your results (if needed)
        with open(file_name, 'a', encoding='utf-8') as f:
            for doc in page['hits']['hits']:
                f.write(json.dumps(doc['_source'], ensure_ascii=False) + "\n")
        # Fetch the next page
        page = es.scroll(scroll_id=sid, scroll='2m')
        # Update the scroll ID and size
        sid = page['_scroll_id']
        scroll_size = len(page['hits']['hits'])

In [3]:
index_name = "logs-ms_signins_interactive"
# Fetch all documents
fetch_all_docs(index_name, "logs-ms_signins_noninteractive.json")

/tmp/ipykernel_33737/2289539765.py:3: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  page = es.search(
